In [84]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
import threading
import time

from datetime import datetime

## Group Assignment
### Team Number: 08
### Team Member Names: Richard Meng, Avaansh Nanda, Devesh Kotak
### Team Strategy Chosen: Market Beat

We first start by reading the CSV file of tickers and processing the tickers as a list for easy iteration.

In [85]:
# read in the csv file and set the header to none, otherwise it will treat the first ticker as the header
tickers_csv = pd.read_csv("Tickers_Example.csv", header=None)
# turn the csv dataframe into a list
tickers = list(tickers_csv[0])
tickers

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AGN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BB.TO',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CELG',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTN',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

Below are some helper functions to help assist filtering our stock data using data threading. 

In [86]:
#  We will be filtering 40 tickers. Each thread will handle 5 tickers which download from yfinance simultaneously, speeding up the process.
START_DATE = "2024-10-01"
END_DATE = "2025-09-30"

# helper function to download stock data for a subset of tickers, index is between (0, number_of_threads-1)
def download_yf(ticker_subset, result_df, index):
    data = {}
    # looping through each ticker and downloading the data from yfinance, and storing in 
    for ticker in ticker_subset:
        data[ticker] = yf.download(ticker, START_DATE, END_DATE)
    result_df[index] = data 
    return result_df # returning the dataframe with updated values 

# helper function to create & start a thread to download 5 tickers' stock data from yfinance
def start_threading(five_tickers_array):
    # creating 8 threads in the for loop
    threading.Thread(target=download_yf, args=(five_tickers_array)).start()


    

Next, we begin filtering the stocks. We first must check that they are Canadian or American, which can be done by checking their currency. Then we check if the company is actually listed on its respective market. Lastly, we check the daily average volume (excluding months with less than 18 trading days) and exclude stocks with less than 5000 daily average volume.

In [87]:
# function to check if a singular ticker is valid, then add it to an array of results at its respective index
# this is the helper function for the thread

def valid_ticker(ticker, results, index):
    try:
        # store the ticker as a yf.Ticker to find info on currency and sector
        yfTicker = yf.Ticker(ticker)
        # store data to get trading volume + loop through the dates to find number of trading days per month
        ticker_data = yf.download(ticker, start=START_DATE, end=END_DATE, auto_adjust=False, progress=False)
        # use time.sleep so yFinance does not time out and limit our request rate due to threading
        time.sleep(2)
        # store the currency
        currency = yfTicker.fast_info.get('currency')
        # store the sector
        sector = yfTicker.info.get('sector')
        # again use time.sleep after each yFinance call so it does not limit our request rate
        time.sleep(2)
        # create a date range with the start of each month in range
        month_starts = pd.date_range(start=START_DATE, end=END_DATE, freq="MS")
        # create a date range with the end of each month in range
        month_ends = pd.date_range(start=START_DATE, end=END_DATE, freq="ME")
        # initialize current month index to 0 and num trading days equal to 0
        cur_month_idx = 0
        num_trading_days = 0
        # keep track of the tickers total volume and number of total trading days to find avg daily volume
        total_vol = 0
        total_trading_days = 0
        # initialize date index
        date_idx = 0
        dates = ticker_data.index
        # loop through each date in the ticker data
        while date_idx < len(dates) and cur_month_idx < len(month_starts):
            date = dates[date_idx]
           # if the date is less than the end date of current month it is in
            if date <= month_ends[cur_month_idx]:
                # first add one to the number of trading days
                num_trading_days += 1
                total_trading_days += 1
                # if it is greater than 18, now we check the daily average volume
                #    if its less than 5000 don't add this stock to valid stocks.
                # otherwise, if the number of trading days isn't greater than 18
                #    continue on as then we'll exclude it or wait until it goes above 18
                if num_trading_days >= min_trading_days:
                    ticker_month_data = yf.download(ticker, start=month_starts[cur_month_idx], end=month_ends[cur_month_idx], auto_adjust=False, progress=False)
                    # again use time.sleep to allow our yFinance requests to go through
                    time.sleep(2)
                    # add to total volume
                    total_vol += ticker_month_data["Volume"].sum().iloc[0]
                    # find the current index of the last date, side="right" gives us the index after the last day of the month's index
                    date_idx = np.searchsorted(dates, month_ends[cur_month_idx], side="right")

            # now we have surpassed the month end and will go to the next month       
            else:
                cur_month_idx += 1
                num_trading_days = 0

            date_idx+=1
        # calculate average daily volume
        avg_daily_vol = total_vol / total_trading_days if total_trading_days > 0 else 0
        # if the average daily volume is less than the minimum or the currency is invalid, don't add the ticker (store it as None).
        if avg_daily_vol<min_volume or not(currency in valid_currencies):
            results[index] = None
            print(f"removed {ticker}")
        # otherwise, if it's valid, then add the key pair ticker, (sector, currency) to valid_tickers for easy access to a ticker's info.
        else:
            results[index] = (ticker, sector, currency)
    # the except will catch delisted stocks and exclude them from valid_tickers (store it as None)
    except Exception as e:
        print(f"{ticker} is delisted {e}")
        results[index] = None

# function for threading to speed up the process
def filter_tickers(tickers):
    # initialize an list threads to store our threads and valid_tickers to get the return value of the valid_ticker function
    threads = []
    valid_tickers = [""] * len(tickers)

    # loop through the list of possible tickers and get the index and actual ticker
    for index, ticker in enumerate(tickers):
        # start a thread using the valid_ticker function and pass in its respective arguments
        t = threading.Thread(target=valid_ticker, args=(ticker, valid_tickers, index))
        # start the thread and append it to the threads list
        t.start()
        threads.append(t)

    # loop through each thread and join them
    for thread in threads:
        thread.join()

    # use a list comprehension to add to valid_tickers if the ticker is not None.
    return [v for v in valid_tickers if v]

# store the valid_tickers in a list
valid_tickers = filter_tickers(tickers)
# initalize a dictionary to store elements as ticker : (sector, currency)
ticker_dict = {}
# add all valid tickers to the dictionary
for ticker_tup in valid_tickers:
    ticker_dict[ticker_tup[0]] = ticker_tup[1::]


$RTN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


RTN is delisted 'currency'


$AGN: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")


AGN is delisted 'currency'
MON is delisted 'currency'
CELG is delisted 'currency'


Here is the list of all tickers that follow the rules and their respective information stored in a dictionary

In [88]:
print(len(ticker_dict.keys()), len(tickers))
ticker_dict

37 41


{'AAPL': ('Technology', 'USD'),
 'ABBV': ('Healthcare', 'USD'),
 'ABT': ('Healthcare', 'USD'),
 'ACN': ('Technology', 'USD'),
 'AIG': ('Financial Services', 'USD'),
 'AMZN': ('Consumer Cyclical', 'USD'),
 'AXP': ('Financial Services', 'USD'),
 'BA': ('Industrials', 'USD'),
 'BAC': ('Financial Services', 'USD'),
 'BB.TO': ('Technology', 'CAD'),
 'BIIB': ('Healthcare', 'USD'),
 'BK': ('Financial Services', 'USD'),
 'BLK': ('Financial Services', 'USD'),
 'BMY': ('Healthcare', 'USD'),
 'C': ('Financial Services', 'USD'),
 'CAT': ('Industrials', 'USD'),
 'CL': ('Consumer Defensive', 'USD'),
 'KO': ('Consumer Defensive', 'USD'),
 'LLY': ('Healthcare', 'USD'),
 'LMT': ('Industrials', 'USD'),
 'MO': ('Consumer Defensive', 'USD'),
 'MRK': ('Healthcare', 'USD'),
 'PEP': ('Consumer Defensive', 'USD'),
 'PFE': ('Healthcare', 'USD'),
 'PG': ('Consumer Defensive', 'USD'),
 'PM': ('Consumer Defensive', 'USD'),
 'PYPL': ('Financial Services', 'USD'),
 'QCOM': ('Technology', 'USD'),
 'RY.TO': ('Financi

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.
- Richard Meng
- Avaansh Nanda
- Devesh Kotak